<a href="https://colab.research.google.com/github/kamalahmadov474/LLM-based-NLP/blob/main/A_Retrieval_Augmented_Generation_Banking_Chatbot_using_LLaMA_3_2_and_FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pull LLaMA 3.2 Model

In [ ]:
!ollama pull llama3.2

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest 
pulling dde5aa3fc5ff: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 2.0 GB                         
pulling 966de95ca8a6: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.4 KB                         
pulling fcc5a6bec9da: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 7.7 KB                         
pulling a70ff7e570d9: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 6.0 KB                         
pulling 56bb8bd477a5: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�   96 B                         
pulling 34bb5ab01051: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  561 B                         
verifying sha256 digest 
writin

Install Required Libraries

In [ ]:
!pip install faiss-cpu sentence-transformers openai pandas

Import Libraries

In [ ]:
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer
from openai import OpenAI

Set Model Name

In [ ]:
MODEL = "llama3.2"
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

Prepare Sample Data

In [ ]:
data = {
    "question": [
        "What is the minimum balance for a savings account?",
        "How do I apply for a home loan?",
        "What are the charges for international transfers?",
        "How can I block my debit card?",
        "What documents are needed to open a new account?",
        "How can I reset my online banking password?",
        "What are the working hours of the bank branches?",
        "How do I check my account statement?",
        "Can I get a personal loan online?",
        "How do I update my contact information?"
    ],
    "answer": [
        "The minimum balance for a savings account is $500.",
        "You can apply for a home loan by filling out the online application form or visiting our branch.",
        "International transfer fees vary, typically $25 per transaction.",
        "You can block your debit card via online banking, mobile app, or calling customer support.",
        "To open a new account, you need a government-issued ID, proof of address, and an initial deposit.",
        "You can reset your online banking password through the 'Forgot Password' option on the login page.",
        "Bank branches are open from 9:00 AM to 5:00 PM, Monday to Friday.",
        "You can check your account statement through online banking or request it at a branch.",
        "Yes, personal loans can be applied for online through the bank's website.",
        "You can update your contact information by logging into online banking or visiting your nearest branch."
    ]
}

df = pd.DataFrame(data)

Initialize Embedding Model

In [ ]:
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
question_embeddings = embedding_model.encode(df["question"].tolist(), convert_to_numpy=True, show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Set Embedding Dimensions

In [ ]:
embedding_dim = question_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(question_embeddings)

Define RAG Query Function

In [ ]:
def rag_query(user_question, top_k=2):
    # 1. Embed the user question
    q_emb = embedding_model.encode([user_question], convert_to_numpy=True)

    # 2. Retrieve top_k similar FAQ answers
    D, I = index.search(q_emb, top_k)
    context = "\n".join([df.iloc[i]["answer"] for i in I[0]])

    # 3. Prepare prompt for Llama 3.2
    prompt = f"Use the following context to answer the question.\nContext: {context}\nQuestion: {user_question}\nAnswer:"

    # 4. Get completion from Ollama Llama 3.2
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

Sample Questions List

In [ ]:
questions = [
    "How can I block my debit card quickly?",
    "What is the minimum amount I must keep in my savings account?",
    "Which documents do I need to open a new bank account?",
    "How do I reset my online banking password?",
    "Can I apply for a personal loan online?",
    "What are the bank branch working hours?",
    "How do I check my account statement?",
    "How can I update my contact information?",
    "How do I apply for a home loan?",
    "What are the charges for international transfers?"
]

for q in questions:
    print(f"User Question: {q}")
    print(f"RAG Answer: {rag_query(q)}\n")

User Question: How can I block my debit card quickly?
RAG Answer: To block your debit card quickly, you can call customer support.

User Question: What is the minimum amount I must keep in my savings account?
RAG Answer: The minimum amount you must keep in your savings account is $500.

User Question: Which documents do I need to open a new bank account?
RAG Answer: To open a new account, you will need two documents:

1. A government-issued ID
2. Proof of address

You can reset your online banking password using the 'Forgot Password' option on the login page if you cannot remember it.

User Question: How do I reset my online banking password?
RAG Answer: To reset your online banking password, click on the 'Forgot Password' option on the login page and follow the prompts to create a new password.

User Question: Can I apply for a personal loan online?
RAG Answer: Yes, you can apply for a personal loan online.

User Question: What are the bank branch working hours?
RAG Answer: The bank b